# ReGenNexus Core - Basic Demo

This notebook demonstrates the basic functionality of ReGenNexus Core, a Universal Agent Protocol for secure communication between agents, devices, and services.

## Setup

First, let's install ReGenNexus directly from GitHub:

In [ ]:
# Clone the repository first
!git clone https://github.com/ReGenNow/ReGenNexus.git
%cd ReGenNexus
# Install in development mode
!pip install -e .
# Install additional dependencies
!pip install ipywidgets matplotlib numpy nest_asyncio
# Return to the parent directory
%cd ..

In [ ]:
import asyncio
import json
import time
import uuid
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import nest_asyncio
import matplotlib.pyplot as plt
import numpy as np

# Apply nest_asyncio to allow asyncio to work in Jupyter
nest_asyncio.apply()

In [ ]:
# Import ReGenNexus Core modules
from regennexus.protocol.client import Client
from regennexus.protocol.message import Message
from regennexus.protocol.protocol_core import Protocol
from regennexus.security.security import SecurityManager

## Create Demo Environment

Let's set up a simple demo environment with two agents communicating through ReGenNexus:

In [ ]:
# Create output area for logs
log_output = widgets.Output()
display(HTML("<h2>ReGenNexus Core Demo</h2>"))
display(HTML("<h3>Log Output:</h3>"))
display(log_output)

In [ ]:
# Create clients for two agents
agent1_client = None
agent2_client = None

async def setup_demo():
    """Set up the demo environment."""
    global agent1_client, agent2_client
    
    with log_output:
        print("Setting up demo environment...")
        
        # Create security manager for each agent
        security1 = SecurityManager()
        security2 = SecurityManager()
        
        # Initialize security
        await security1.initialize()
        await security2.initialize()
        
        # Create clients
        agent1_client = Client(
            entity_id="agent1",
            entity_type="agent",
            security_manager=security1
        )
        
        agent2_client = Client(
            entity_id="agent2",
            entity_type="agent",
            security_manager=security2
        )
        
        # Initialize clients
        await agent1_client.initialize()
        await agent2_client.initialize()
        
        print("Demo environment ready!")
        print(f"Agent 1 ID: {agent1_client.entity_id}")
        print(f"Agent 2 ID: {agent2_client.entity_id}")

In [ ]:
# Message handlers
async def agent1_message_handler(message):
    """Handle messages received by Agent 1."""
    with log_output:
        print(f"Agent 1 received: {message.content}")
        
        # Respond to the message
        if "hello" in message.content.lower():
            response = "Hello to you too, Agent 2!"
            await agent1_client.send_message(message.sender_id, response)
            print(f"Agent 1 sent: {response}")

async def agent2_message_handler(message):
    """Handle messages received by Agent 2."""
    with log_output:
        print(f"Agent 2 received: {message.content}")
        
        # Respond to the message
        if "task" in message.content.lower():
            response = "I'll get started on that task right away!"
            await agent2_client.send_message(message.sender_id, response)
            print(f"Agent 2 sent: {response}")

In [ ]:
# Register message handlers
async def register_handlers():
    """Register message handlers for both agents."""
    with log_output:
        print("Registering message handlers...")
        
        # Register handlers
        agent1_client.register_message_handler(agent1_message_handler)
        agent2_client.register_message_handler(agent2_message_handler)
        
        print("Message handlers registered")

In [ ]:
# Send test messages
async def send_test_messages():
    """Send test messages between agents."""
    with log_output:
        print("\nSending test messages...")
        
        # Agent 1 sends a greeting to Agent 2
        await agent1_client.send_message(agent2_client.entity_id, "Hello, Agent 2!")
        print("Agent 1 sent: Hello, Agent 2!")
        
        # Wait a moment for processing
        await asyncio.sleep(1)
        
        # Agent 1 sends a task to Agent 2
        await agent1_client.send_message(
            agent2_client.entity_id, 
            "I need you to complete this task for me."
        )
        print("Agent 1 sent: I need you to complete this task for me.")
        
        # Wait for processing
        await asyncio.sleep(1)
        
        print("Test messages sent and processed")

In [ ]:
# Visualize message flow
async def visualize_message_flow():
    """Create a visualization of message flow between agents."""
    with log_output:
        print("\nVisualizing message flow...")
        
        # Create a simple visualization
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # Agent positions
        agent1_pos = (0.2, 0.5)
        agent2_pos = (0.8, 0.5)
        
        # Draw agents
        ax.plot(agent1_pos[0], agent1_pos[1], 'o', markersize=20, color='blue')
        ax.plot(agent2_pos[0], agent2_pos[1], 'o', markersize=20, color='green')
        
        # Label agents
        ax.text(agent1_pos[0], agent1_pos[1]-0.1, "Agent 1", ha='center', fontsize=12)
        ax.text(agent2_pos[0], agent2_pos[1]-0.1, "Agent 2", ha='center', fontsize=12)
        
        # Draw message arrows
        # Message 1: Agent 1 -> Agent 2 (Hello)
        ax.arrow(agent1_pos[0]+0.02, agent1_pos[1], 
                 agent2_pos[0]-agent1_pos[0]-0.04, 0, 
                 head_width=0.03, head_length=0.03, fc='blue', ec='blue')
        ax.text(0.5, 0.6, "Hello, Agent 2!", ha='center', fontsize=10)
        
        # Message 2: Agent 2 -> Agent 1 (Response)
        ax.arrow(agent2_pos[0]-0.02, agent2_pos[1]-0.05, 
                 agent1_pos[0]-agent2_pos[0]+0.04, 0, 
                 head_width=0.03, head_length=0.03, fc='green', ec='green')
        ax.text(0.5, 0.4, "Hello to you too, Agent 1!", ha='center', fontsize=10)
        
        # Message 3: Agent 1 -> Agent 2 (Task)
        ax.arrow(agent1_pos[0]+0.02, agent1_pos[1]+0.05, 
                 agent2_pos[0]-agent1_pos[0]-0.04, 0, 
                 head_width=0.03, head_length=0.03, fc='blue', ec='blue')
        ax.text(0.5, 0.7, "I need you to complete this task for me.", ha='center', fontsize=10)
        
        # Message 4: Agent 2 -> Agent 1 (Task response)
        ax.arrow(agent2_pos[0]-0.02, agent2_pos[1]+0.1, 
                 agent1_pos[0]-agent2_pos[0]+0.04, 0, 
                 head_width=0.03, head_length=0.03, fc='green', ec='green')
        ax.text(0.5, 0.3, "I'll get started on that task right away!", ha='center', fontsize=10)
        
        # Set limits and remove axes
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.set_title("ReGenNexus Message Flow")
        ax.axis('off')
        
        plt.show()
        print("Message flow visualization complete")

In [ ]:
# Clean up demo
async def cleanup_demo():
    """Clean up the demo environment."""
    global agent1_client, agent2_client
    
    with log_output:
        print("\nCleaning up demo environment...")
        
        # Shut down clients
        if agent1_client:
            await agent1_client.shutdown()
        
        if agent2_client:
            await agent2_client.shutdown()
        
        print("Demo environment cleaned up")

In [ ]:
# Run the demo
async def run_demo():
    """Run the complete demo."""
    try:
        # Set up the demo
        await setup_demo()
        
        # Register message handlers
        await register_handlers()
        
        # Send test messages
        await send_test_messages()
        
        # Visualize message flow
        await visualize_message_flow()
        
    except Exception as e:
        with log_output:
            print(f"Error in demo: {str(e)}")
    finally:
        # Clean up
        await cleanup_demo()

In [ ]:
# Execute the demo
asyncio.run(run_demo())

## Interactive Demo Controls

Let's create some interactive controls to send custom messages between agents:

In [ ]:
# Reset the demo environment
asyncio.run(setup_demo())
asyncio.run(register_handlers())

In [ ]:
# Create UI elements
message_input = widgets.Text(
    value='',
    placeholder='Type a message',
    description='Message:',
    disabled=False
)

sender_dropdown = widgets.Dropdown(
    options=['Agent 1', 'Agent 2'],
    value='Agent 1',
    description='Sender:',
    disabled=False,
)

send_button = widgets.Button(
    description='Send Message',
    disabled=False,
    button_style='', 
    tooltip='Send the message',
    icon='paper-plane'
)

clear_log_button = widgets.Button(
    description='Clear Log',
    disabled=False,
    button_style='', 
    tooltip='Clear the log output',
    icon='trash'
)

# Define button callbacks
def on_send_button_clicked(b):
    message = message_input.value
    sender = sender_dropdown.value
    
    if not message:
        return
    
    async def send_message():
        if sender == 'Agent 1':
            await agent1_client.send_message(agent2_client.entity_id, message)
            with log_output:
                print(f"Agent 1 sent: {message}")
        else:
            await agent2_client.send_message(agent1_client.entity_id, message)
            with log_output:
                print(f"Agent 2 sent: {message}")
    
    asyncio.ensure_future(send_message())
    message_input.value = ''

def on_clear_log_button_clicked(b):
    log_output.clear_output()

# Connect callbacks
send_button.on_click(on_send_button_clicked)
clear_log_button.on_click(on_clear_log_button_clicked)

# Display UI
display(widgets.HBox([sender_dropdown, message_input]))
display(widgets.HBox([send_button, clear_log_button]))

## Conclusion

This notebook demonstrated the basic functionality of ReGenNexus Core:

1. Setting up secure communication between agents
2. Sending and receiving messages
3. Handling messages with custom logic
4. Visualizing message flow

ReGenNexus provides a secure, flexible framework for agent communication that can be extended for various use cases including IoT, robotics, and multi-agent systems.

In [ ]:
# Clean up when done
asyncio.run(cleanup_demo())